In [1]:
import pickle
import itertools
import open3d as o3d
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import pathlib

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [40]:
import pickle

with open('pt_coord.pickle', 'rb') as f:
    pt_list = pickle.load(f)

In [41]:
pt_list_np = np.asarray(pt_list)
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(pt_list_np)

In [42]:
x_sd = np.sqrt(np.sum(np.square(pt_list_np[:,0] - np.average(pt_list_np[:,0])))/pt_list_np[:,0].size)
x_ave = np.average(pt_list_np[:,0])
x_idx = np.all(np.stack((pt_list_np[:,0] < x_ave + 3 * x_sd, pt_list_np[:,0] > x_ave - 3 * x_sd),axis=1),axis=1)

y_sd = np.sqrt(np.sum(np.square(pt_list_np[:,1] - np.average(pt_list_np[:,1])))/pt_list_np[:,1].size)
y_ave = np.average(pt_list_np[:,1])
y_idx = np.all(np.stack((pt_list_np[:,1] < y_ave + 3 * y_sd, pt_list_np[:,1] > y_ave - 3 * y_sd),axis=1),axis=1)

z_sd = np.sqrt(np.sum(np.square(pt_list_np[:,2] - np.average(pt_list_np[:,2])))/pt_list_np[:,2].size)
z_ave = np.average(pt_list_np[:,2])
z_idx = np.all(np.stack((pt_list_np[:,2] < z_ave + 3 * z_sd, pt_list_np[:,2] > z_ave - 3 * z_sd),axis=1),axis=1)

In [43]:
pt_list_np = pt_list_np[np.all(np.stack((x_idx, y_idx, z_idx),axis=1), axis=1)]
(np.array([True, False]))
pcd.points = o3d.utility.Vector3dVector(pt_list_np)

In [44]:
pts = np.asarray(pcd.points)
pli = np.array([True, False,False])
idx = np.tile(pli, pts.shape[0]//3)
idx = np.append(idx, [pli[i] for i in range(pts.shape[0]%3)])

pcd.points = o3d.utility.Vector3dVector(pts[idx])

In [45]:
o3d.visualization.draw_geometries([pcd])

In [46]:
# クラスタリング
with o3d.utility.VerbosityContextManager(o3d.utility.VerbosityLevel.Debug) as cm:
    labels = np.array(pcd.cluster_dbscan(eps=0.1, min_points=100))

max_label = labels.max()
print(f"point cloud has {max_label + 1} clusters")
colors = plt.get_cmap("tab20")(labels / (max_label if max_label > 0 else 1))
colors[labels < 0] = 0
pcd.colors = o3d.utility.Vector3dVector(colors[:, :3])
o3d.visualization.draw_geometries([pcd])

[Open3D DEBUG] Precompute neighbors.
[Open3D DEBUG] Done Precompute neighbors.
[Open3D DEBUG] Compute Clusters
[Open3D DEBUG] Done Compute Clusters: 7
point cloud has 7 clusters


In [61]:
pcd_np = np.asarray(pcd.points)


In [62]:
noise_list = []
for i in range(np.max(labels)+1):
    if np.sum(labels == i)>100:
        noise_list.append(i)
noise_list = np.asarray(noise_list)

In [63]:
t_list = np.zeros(len(labels),dtype=np.bool8)
for i in noise_list:
    t_list += labels == i

In [64]:
denoise_pcd_np = pcd_np[t_list]
denoise_colors = colors[t_list]
denoise_labels = labels[t_list]

In [65]:
denoise_pcd = o3d.geometry.PointCloud()
denoise_pcd.points = o3d.utility.Vector3dVector(denoise_pcd_np)
#denoise_pcd.colors = o3d.utility.Vector3dVector(denoise_colors[:, :3])

In [66]:
o3d.visualization.draw_geometries([denoise_pcd])

In [67]:
# クラスタリング
with o3d.utility.VerbosityContextManager(o3d.utility.VerbosityLevel.Debug) as cm:
    labels = np.array(denoise_pcd.cluster_dbscan(eps=0.1, min_points=100))

max_label = labels.max()
print(f"point cloud has {max_label + 1} clusters")
colors = plt.get_cmap("tab20")(labels / (max_label if max_label > 0 else 1))
colors[labels < 0] = 0
denoise_pcd.colors = o3d.utility.Vector3dVector(colors[:, :3])
o3d.visualization.draw_geometries([denoise_pcd])

[Open3D DEBUG] Precompute neighbors.
[Open3D DEBUG] Done Precompute neighbors.
[Open3D DEBUG] Compute Clusters
[Open3D DEBUG] Done Compute Clusters: 7
point cloud has 7 clusters
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: 要求された変換操作はサポートされていません。 


In [68]:
o3d.io.write_point_cloud("denoise.ply", denoise_pcd)

True